In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn

In [3]:
class CustomDataset(Dataset):
    def __init__(self, data_file, labels_file):
        with open(data_file, 'r') as file:
            self.data = []
            for line in file:
                numbers = list(map(float, line.split()))  # Read and split numbers
                self.data.append(numbers)

        with open(labels_file, 'r') as file:
            self.labels = []
            for line in file:
                label = list(map(float, line.split()))
                label = label[2:4]
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.float) 
        }
        return sample

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, predicted, target):
        loss_function = nn.MSELoss()  # Using Mean Squared Error loss
        loss_1 = loss_function(predicted[0], target[0])  # Loss for first output
        loss_2 = loss_function(predicted[1], target[1])  # Loss for second output
        total_loss = loss_1 # + loss_2  # Combine the losses

        return total_loss

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(5, 8),
      nn.ReLU(),
      nn.Linear(8, 4),
      nn.ReLU(),
      nn.Linear(4, 2)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [ ]:

data_file_path = 'dataset/acoustic_data.txt'
labels_file_path = 'dataset/geometry_data.txt'
dataset = CustomDataset(data_file_path, labels_file_path)
# Define the sizes for the training and test sets
train_size = int(0.8 * len(dataset))  # 80% training
test_size = len(dataset) - train_size  # 20% test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# load training set and test set
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Initialize the MLP
mlp = MLP()

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

# Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_loader, 0):
    
        # Get inputs
        inputs = data["data"]
        targets = data["label"]

        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = mlp(inputs)
        
        # Compute loss
        criterion = CustomLoss()
        loss = criterion(outputs, targets)
        
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                    (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')


In [ ]:

# validation process
accuracy = list()

for i, data in enumerate(test_dataset, 0):
    # Get inputs
    inputs = data["data"]
    targets = data["label"]
    outputs = mlp(inputs)
    # calculate accuracy
    if i == 0:
        avg_accuracy = abs(outputs - targets) / targets * 100
    else:
        avg_accuracy += abs(outputs - targets) / targets * 100

    accuracy.append(abs(outputs - targets) / targets * 100)
avg_accuracy /= test_size
print(avg_accuracy)
